In [42]:
import serial
import time

In [355]:
# Enable Serial Communication
ser = serial.Serial("/dev/ttyUSB0", baudrate=115200, timeout=1)

In [356]:
def getResult(ser, n=4):
    ser.readline() # \r\n
    o = ser.readline() # Actual output
    while n > 2:
        ser.readline();
        n -= 1
    return o

class gps_sim908:
    def __init__(self, sfile):
        self.sfile = sfile
        self.sfile.write(b'ATE0'+b'\r')      # Disable the Echo
        self.sfile.flush()
        time.sleep(1)
        self.sfile.read_all()
        
        self.sfile.write(b'AT+CGPSPWR?' + b'\r')
        self.sfile.flush()
        o = getResult(self.sfile)
        print(o)
        
        if b'CGPSPWR: 1' not in o:
            print('Powering on GPS')
            self.sfile.write(b'AT+CGPSPWR=1' + b'\r')
            self.sfile.flush()
            print(getResult(self.sfile))
            time.sleep(1)

            self.sfile.write(b'AT+CGPSRST=0' + b'\r')
            self.sfile.flush()
            print(getResult(self.sfile))
            time.sleep(1)
            
        self.sfile.write(b'AT+CGPSINF=0' + b'\r')
        self.sfile.flush()
        print(getResult(self.sfile, 3))
        
        self.sfile.write(b'AT+CGPSSTATUS?' + b'\r')
        self.sfile.flush()
        o = getResult(self.sfile)
        print(o)
        
        while b'Fix' not in o:
            print('Attempting to get GPS Fix...')
            time.sleep(10)
            self.sfile.write(b'AT+CGPSSTATUS? ' + b'\r')
            self.sfile.flush()
            o = getResult(self.sfile)
            print(o)
        print('Got GPS Fix Successfully!')
    def getRaw(self, inf = b'32'):
        self.sfile.write(b'AT+CGPSINF=' + inf + b'\r')
        #self.sfile.flush()
        o = getResult(self.sfile, 3)
        return o
        

In [357]:
g = gps_sim908(ser)

b'+CGPSPWR: 1\r\n'
b'0,7909.751296,1258.317141,160.486588,20190902101057.000,4347,13,0.000000,339.348022\r\n'
b'+CGPSSTATUS: Location 3D Fix\r\n'
Got GPS Fix Successfully!


In [359]:
g.getRaw(b'4')

b'4,1258.317141,N,7909.751296,E,101102.000,A,A\r\n'

In [360]:
n = 1000
l = range(0,n)
j = time.time()
for i in l:
    g.getRaw(b'4')
v = time.time() - j
print(v)
print("Frequency = ", n/v)

16.03750991821289
Frequency =  62.35381958294889


In [259]:
h = b'0,7909.746663,1258.403408,76.240837,20190902090034.000,89,6,0.000000,0.000000\r\n'

In [261]:
h.decode('ascii')

'0,7909.746663,1258.403408,76.240837,20190902090034.000,89,6,0.000000,0.000000\r\n'

In [338]:
ser.write(b'AT+IPR=115200'+b'\r')      # Disable the Echo
rcv = ser.readline()
print(rcv)
time.sleep(1)


b'\r\n'


In [339]:
ser.read_all()

b'OK\r\n'

In [272]:
ser.write(b'AT+CGPSINF=32'+b'\r')
ser.flush()
#ser.flushOutput()
rcv = ser.readline()
print(rcv)
rcv = ser.readline()
print(rcv)
print(ser.readline())
ser.readline()

b'\r\n'
b'+CGPSSTATUS: Location 3D Fix\r\n'
b'\r\n'


b'OK\r\n'

In [231]:
ser.read_all

b''

In [ ]:
ser.